In [4]:
!pip install regex
!pip install pandas
!pip install nltk

In [5]:
import re
import pandas as pd
from nltk import flatten
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

def find_thr( start , end, sequence ):
    """funtion to look for G2 box """
    
    pos = [m.start() for m in re.finditer('T', sequence[start:end])]
    
    return(pos)


In [0]:


def create_table(path1, path2):
    df1 = pd.read_csv(path1)
    df2 = pd.read_excel(path2)
    df1 = df1.rename(columns={'ProteinID':'Entry'})
    
    df = pd.merge(df1[["Entry","Position","Position.1"]], df2[["Entry","Sequence" ]], on='Entry')
    
    return df


In [0]:

def flatten_dict(thr_dict):
    new_thr_dict = dict()
    for key, value in thr_dict.items():
        #print(key)
        #print(value)
        new_thr_dict[key] = flatten(value)
    return new_thr_dict


In [0]:
sheet_path= "/content/drive/My Drive/Colab Notebooks/GProtein_Project/sheet.csv"
data_sheet = pd.read_csv(sheet_path)

In [0]:
data_sheet.head()

In [0]:
family = data_sheet.drop_duplicates(['Protein'])[['Protein', 'in_path_new', 'out_path_new', "Thr_out"]]
family = family.reset_index()

In [0]:
family.head()

In [0]:

for i in range(family.shape[0]):
    
    thr_dict = dict()
    
    in_path_idx = family.loc[i, 'out_path_new']
    in_path_seq = family.loc[i, 'in_path_new']
    
    out_path = family.loc[i, 'Thr_out']
    
    df = create_table(in_path_idx, in_path_seq)
    
    
    for j in range(df.shape[0]):
        start = df.loc[j, 'Position']+7
        end = df.loc[j, 'Position.1']
        seq = df.loc[j, 'Sequence']
        
        protein = df.loc[j, 'Entry']
        
        pos = find_thr(start , end, seq)
        
        if(protein not in thr_dict):
            thr_dict[protein] = list()
            thr_dict[protein].append(pos)
        else:
            thr_dict[protein].append(pos)
    
    thr_dict = flatten_dict(thr_dict)
    table = pd.DataFrame.from_dict(thr_dict, orient='index') 
    table.to_csv(out_path)